In [114]:
### Load the houseprices data from Thinkful's database.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNetCV

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
house_df = pd.read_sql_query('select * from houseprices',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()

# fix for an issue with centered not properly loading with current version

import  scipy.signal.signaltools

def _centered(arr, newsize):
    # Return the center newsize portion of the array.
    newsize = np.asarray(newsize)
    currsize = np.array(arr.shape)
    startind = (currsize - newsize) // 2
    endind = startind + newsize
    myslice = [slice(startind[k], endind[k]) for k in range(len(endind))]
    return arr[tuple(myslice)]

scipy.signal.signaltools._centered = _centered

import statsmodels.api as sm
from statsmodels.tools.eval_measures import mse, rmse

In [160]:
### Do data cleaning, exploratory data analysis, and feature engineering. You can use your previous work in this module. But, 
### make sure that your work is satisfactory.

### Now, split your data into train and test sets where 20% of the data resides in the test set.

house_df = pd.concat([house_df,pd.get_dummies(house_df.centralair, prefix = "centralair", drop_first = True)], axis = 1)
dummy_column_names = list(pd.get_dummies(house_df.centralair, prefix = "centralair", drop_first = True).columns)
house_df['totalsf'] = house_df['totalbsmtsf'] + house_df['firstflrsf'] + house_df['secondflrsf']
Y = np.log1p(house_df['saleprice'])
X = house_df[['overallqual', 'grlivarea', 'garagecars', 'totalsf'] + dummy_column_names]
X = sm.add_constant(X)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 37)
alphas = [np.power(10.0,p) for p in np.arange(-10,40,1)]

In [157]:
###############################################################################################################################
###############################################################################################################################
###############################################################################################################################
### 6. Load new morgate rate data and groupy by year and add it to the house_df dataset; years 2006-2010
### historicalweeklydata - Full History.csv

mortage_rate = pd.read_csv('historicalweeklydata - Full History.csv')
mortage_rate = mortage_rate[mortage_rate['Year'] >= '2006']
mortage_rate = mortage_rate[mortage_rate['Year'] <= '2010']
average_rate = mortage_rate.groupby('Year', as_index = False)['30 yr FRM'].mean()
average_rate['Year'] = average_rate['Year'].astype(int)

# ew horrible but the show must go on

rate06 = round(average_rate['30 yr FRM'][0], 3)
rate07 = round(average_rate['30 yr FRM'][1], 3)
rate08 = round(average_rate['30 yr FRM'][2], 3)
rate09 = round(average_rate['30 yr FRM'][3], 3)
rate10 = round(average_rate['30 yr FRM'][4], 3)

for i, row, in enumerate(house_df):
    hYear = house_df['yrsold'][i]
    if hYear == 2006:
        house_df['m_rate'][i] = rate06
    elif hYear == 2007:
        house_df['m_rate'][i] = rate07
    elif hYear == 2008:
        house_df['m_rate'][i] = rate08
    elif hYear == 2009:
        house_df['m_rate'][i] = rate09
    elif hYear == 2010:
        house_df['m_rate'][i] = rate10
    else: 
        house_df['m_rate'][i] = average_rate['30 yr FRM'].mean()
###############################################################################################################################
###############################################################################################################################
###############################################################################################################################

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice,centralair_Y,totalsf,m_rate
0,1,60,RL,65.000,8450,Pave,None,Reg,Lvl,AllPub,...,None,0,2,2008,WD,Normal,208500,1,2566,6.027
1,2,20,RL,80.000,9600,Pave,None,Reg,Lvl,AllPub,...,None,0,5,2007,WD,Normal,181500,1,2524,6.337
2,3,60,RL,68.000,11250,Pave,None,IR1,Lvl,AllPub,...,None,0,9,2008,WD,Normal,223500,1,2706,6.027
3,4,70,RL,60.000,9550,Pave,None,IR1,Lvl,AllPub,...,None,0,2,2006,WD,Abnorml,140000,1,2473,6.413
4,5,60,RL,84.000,14260,Pave,None,IR1,Lvl,AllPub,...,None,0,12,2008,WD,Normal,250000,1,3343,6.027
5,6,50,RL,85.000,14115,Pave,None,IR1,Lvl,AllPub,...,Shed,700,10,2009,WD,Normal,143000,1,2158,5.037
6,7,20,RL,75.000,10084,Pave,None,Reg,Lvl,AllPub,...,None,0,8,2007,WD,Normal,307000,1,3380,6.337
7,8,60,RL,NaN,10382,Pave,None,IR1,Lvl,AllPub,...,Shed,350,11,2009,WD,Normal,200000,1,3197,5.037
8,9,50,RM,51.000,6120,Pave,None,Reg,Lvl,AllPub,...,None,0,4,2008,WD,Abnorml,129900,1,2726,6.027
9,10,190,RL,50.000,7420,Pave,None,Reg,Lvl,AllPub,...,None,0,1,2008,WD,Normal,118000,1,2068,6.027


In [ ]:
### Build several linear regression models including lasso, ridge, or elastic net, and train them in the training set. Use 
### k-fold cross-validation to select the best hyperparameters if your models include one.

### Evaluate your best model on the test set. Per last lesson Lasso seems to be the best of the three.

In [161]:
### OLS 

lrm = LinearRegression()
lrm.fit(X_train, y_train)
y_preds_train = lrm.predict(X_train)
y_preds_test = lrm.predict(X_test)

print("R-squared of the model in the training set is: {}".format(lrm.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in the test set is: {}".format(lrm.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

R-squared of the model in the training set is: 0.8191858742162685
-----Test set statistics-----
R-squared of the model in the test set is: 0.7776690719006185
Mean absolute error of the prediction is: 0.12720268943550664
Mean squared error of the prediction is: 0.034147143979616525
Root mean squared error of the prediction is: 0.18478945851865178
Mean absolute percentage error of the prediction is: 1.0609997089249397


In [158]:
# Lasso

lassoregr = LassoCV(alphas = alphas, cv = 5) 
lassoregr.fit(X_train, y_train)
y_preds_train = lassoregr.predict(X_train)
y_preds_test = lassoregr.predict(X_test)

print("R-squared of the model on the training set is: {}".format(lassoregr.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model on the test set is: {}".format(lassoregr.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

R-squared of the model on the training set is: 0.8191858742162685
-----Test set statistics-----
R-squared of the model on the test set is: 0.7776690719634503
Mean absolute error of the prediction is: 0.12720268941039048
Mean squared error of the prediction is: 0.03414714396996638
Root mean squared error of the prediction is: 0.18478945849254058
Mean absolute percentage error of the prediction is: 1.060999708705768


In [ ]:
# Ridge

ridgeregr = RidgeCV(alphas = alphas, cv = 5) 
ridgeregr.fit(X_train, y_train)
y_preds_train = ridgeregr.predict(X_train)
y_preds_test = ridgeregr.predict(X_test)

print("R-squared of the model on the training set is: {}".format(ridgeregr.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model on the test set is: {}".format(ridgeregr.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

In [163]:
### So far, you have only used the features in the dataset. However, house prices can be affected by many factors like economic 
### activity and the interest rates at the time they are sold. So, try to find some useful factors that are not included in the 
### dataset. Integrate these factors into your model and assess the prediction performance of your model. Discuss the 
### implications of adding these external variables into your model.

# add mortage rate

Y = np.log1p(house_df['saleprice'])
X = house_df[['overallqual', 'grlivarea', 'garagecars', 'totalsf', 'm_rate'] + dummy_column_names]
X = sm.add_constant(X)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 37)
alphas = [np.power(10.0,p) for p in np.arange(-10,40,1)]

In [164]:
lrm = LinearRegression()
lrm.fit(X_train, y_train)
y_preds_train = lrm.predict(X_train)
y_preds_test = lrm.predict(X_test)

print("R-squared of the model in the training set is: {}".format(lrm.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in the test set is: {}".format(lrm.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

R-squared of the model in the training set is: 0.8192349250768427
-----Test set statistics-----
R-squared of the model in the test set is: 0.7770283256713143
Mean absolute error of the prediction is: 0.12738389234334727
Mean squared error of the prediction is: 0.03424555428147371
Root mean squared error of the prediction is: 0.18505554377395375
Mean absolute percentage error of the prediction is: 1.062530966055125


In [ ]:
# While lasso was originally looking to be the model it produced identical results after adding mortage rate into the model
# We then check the second best model from the previous lesson which was LinearRegression. This saw minor improvements. 
# R-squared increases a extremely small amount and we see a slight decrease in mean values. Positive results but ultimatley
# not the kind of impact one is looking for. Mortage rates dont change too drastically year to year so it makes sense
# that they are not going to prosuce meaningful change in short term models that only cover a few year.

# Without mortage rate
#R-squared of the model in the training set is: 0.8191858742162685
#-----Test set statistics-----
#R-squared of the model in the test set is: 0.7776690719006185
#Mean absolute error of the prediction is: 0.12720268943550664
#Mean squared error of the prediction is: 0.034147143979616525
#Root mean squared error of the prediction is: 0.18478945851865178
#Mean absolute percentage error of the prediction is: 1.0609997089249397
    
# With mortage rate
#R-squared of the model in the training set is: 0.8192349250768427
#-----Test set statistics-----
#R-squared of the model in the test set is: 0.7770283256713143
#Mean absolute error of the prediction is: 0.12738389234334727
#Mean squared error of the prediction is: 0.03424555428147371
#Root mean squared error of the prediction is: 0.18505554377395375
#Mean absolute percentage error of the prediction is: 1.062530966055125